In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import numpy as np
import os
import pandas as pd
# function for getting full transcript from vtt
import webvtt


In [ ]:

def get_transcript(filename) -> str:
    vtt = webvtt.read(filename)
    transcript = ""

    lines = []
    for line in vtt:
        lines.extend(line.text.strip().splitlines())

    previous = None
    for line in lines:
        if line == previous:
           continue
        transcript += " " + line
        previous = line

    # print(transcript)
    return(transcript)


In [ ]:

import os
count = 0
for path in os.scandir('subs'):
    count += 1
    print(path.name)

print(count)
df_source = pd.read_csv('youtube_audit_dataset_unique.csv')
num_debunk = 0
num_neutral = 0
num_promote = 0
file_list = os.scandir('subs')

filename_list = []
for file in file_list:
    filename_length = len(file.name)
    # temp = str(file.name[1:(filename_length-7)])
    temp = file.name.split('.')[0]
    filename_list.append(temp)
    # print(temp)


In [ ]:

for idx in df_source.index:
    # print(str(df_source['vid_url'][idx])[-11:])
    if str(df_source['vid_url'][idx])[-11:] in filename_list:
        stance = df_source['normalized_annotation'][idx]
        if stance == 0:
            num_neutral += 1
        elif stance == 1:
            num_debunk += 1
        elif stance == -1:
            num_promote += 1


In [ ]:

print("Neutral: " + str(num_neutral) + "\nPromote: " + str(num_promote) + "\nDebunk: " + str(num_debunk))
finished_df = pd.DataFrame(columns=['annotation', 'stance'])


In [ ]:
for idx in df_source.index:
    if str(df_source['vid_url'][idx])[-11:] in filename_list:
        stance = df_source['normalized_annotation'][idx]

        if stance not in ['-1','0','1']:
            print('ISSUE ' + str(stance))
            break

        vtt_filename = str(df_source['vid_url'][idx])[-11:] + ".vtt"
        vtt_filename = 'subs/' + vtt_filename
        try:
            annotation = get_transcript(vtt_filename)
        except Exception as e:
            print(str(e))
        finished_df.loc[len(finished_df.index)] = [annotation, stance]

finished_df


In [ ]:
finished_df.iloc[5]


In [ ]:
finished_df.to_csv("finished_df_new.csv")